# Day 16 - Filtering and Sorting Data


## Why is Filtering and Sorting Important?
When working with real-world datasets, not all the data may be relevant to your analysis. Filtering helps you focus on the specific subset of data that matters most to your analysis. Sorting, on the other hand, allows you to organize your data to identify trends, compare entries, or prepare it for further analysis.



## Tutorial: Querying DataFrames Based on Conditions
Pandas makes it easy to filter and sort data in a DataFrame using boolean indexing, the `query()` method, and the `sort_values()` function. Let's explore these techniques with practical examples.


### Filtering Data

In [1]:
!pip install pandas

In [3]:
import pandas as pd
# Example DataFrame
data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'Age': [24, 30, 22, 35, 28],
    'Job': ['Engineer', 'Doctor', 'Artist', 'Engineer', 'Doctor']
}
df = pd.DataFrame(data)

# Filtering for rows where Age is greater than 25
filtered_df = df[df['Age'] > 25]
print("Filtered DataFrame (Age > 25):")
print(filtered_df)


Filtered DataFrame (Age > 25):
    Name  Age       Job
1    Bob   30    Doctor
3  David   35  Engineer
4    Eve   28    Doctor


### Using `query()` for Filtering

In [4]:

# Using the query() method
filtered_df = df.query('Age > 25 and Job == "Doctor"')
print("\nFiltered DataFrame (Age > 25 and Job is Doctor):")
print(filtered_df)



Filtered DataFrame (Age > 25 and Job is Doctor):
  Name  Age     Job
1  Bob   30  Doctor
4  Eve   28  Doctor


In [7]:
filtered_df = df.where(df['Age']>25).dropna(how="any")
print("\nFiltered DataFrame (Age > 25):")
print(filtered_df)


Filtered DataFrame (Age > 25):
    Name   Age       Job
1    Bob  30.0    Doctor
3  David  35.0  Engineer
4    Eve  28.0    Doctor


### Sorting Data

In [8]:

# Sorting by Age in ascending order
sorted_df = df.sort_values(by='Age')
print("\nDataFrame sorted by Age (ascending):")
print(sorted_df)

# Sorting by Job and then by Age in descending order
sorted_df = df.sort_values(by=['Job', 'Age'], ascending=[True, False])
print("\nDataFrame sorted by Job and then by Age (descending):")
print(sorted_df)



DataFrame sorted by Age (ascending):
      Name  Age       Job
2  Charlie   22    Artist
0    Alice   24  Engineer
4      Eve   28    Doctor
1      Bob   30    Doctor
3    David   35  Engineer

DataFrame sorted by Job and then by Age (descending):
      Name  Age       Job
2  Charlie   22    Artist
1      Bob   30    Doctor
4      Eve   28    Doctor
3    David   35  Engineer
0    Alice   24  Engineer



## Use Case: Filtering and Sorting ETF Data for Investment Decisions
In this use case, we'll focus on analyzing various ETFs (Exchange-Traded Funds) related to the S&P 500 to help make informed investment decisions. We'll use the Yahoo Finance API to retrieve data on multiple ETFs, calculate key metrics, and sort the ETFs based on relevant financial metrics to aid in our analysis.


### Step 1: Downloading Data for Multiple ETFs

In [9]:

import yfinance as yf
import pandas as pd

# Define a list of ETF tickers related to the S&P 500
etf_tickers = ['SPY', 'IVV', 'VOO', 'SPLG', 'SPYG', 'SPYD', 'SPYV', 'RSP', 'VXF', 'IJR']

# Download historical stock data for these ETFs
etf_data = {}
for ticker in etf_tickers:
    etf_data[ticker] = yf.download(ticker, start='2014-01-01', end='2024-08-19')


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


### Step 2: Calculating ROI Metrics

In [20]:

# Initialize a DataFrame to store ROI and other financial metrics
roi_df = pd.DataFrame(columns=[
    'Ticker', 'YTD_ROI', '1Y_ROI', '5Y_ROI', '10Y_ROI', 
    'P/E Ratio'
])

for ticker, data in etf_data.items():
    # Calculate ROI metrics
    ytd_roi = (data['Adj Close'][-1] / data['Adj Close'][0]) - 1
    one_year_roi = (data['Adj Close'][-1] / data['Adj Close'][-252]) - 1
    five_year_roi = (data['Adj Close'][-1] / data['Adj Close'][-1260]) - 1
    if len(data) >= 2520:
        ten_year_roi = (data['Adj Close'][-1] / data['Adj Close'][-2520]) - 1
    else:
        ten_year_roi = None
    
    # Fetch additional financial metrics from Yahoo Finance
    ticker_info = yf.Ticker(ticker).info
    pe_ratio = ticker_info.get('trailingPE', None)

    # Append the calculated metrics to the DataFrame
    roi_df = roi_df._append({
        'Ticker': ticker,
        'YTD_ROI': ytd_roi,
        '1Y_ROI': one_year_roi,
        '5Y_ROI': five_year_roi,
        '10Y_ROI': ten_year_roi,
        'P/E Ratio': pe_ratio
    }, ignore_index=True)


C:\Users\ricar\AppData\Local\Temp\ipykernel_33812\2722589503.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ytd_roi = (data['Adj Close'][-1] / data['Adj Close'][0]) - 1
C:\Users\ricar\AppData\Local\Temp\ipykernel_33812\2722589503.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  one_year_roi = (data['Adj Close'][-1] / data['Adj Close'][-252]) - 1
C:\Users\ricar\AppData\Local\Temp\ipykernel_33812\2722589503.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value 

### Step 3: Filtering and Sorting the ETFs with Additional Metrics

In [21]:

# Filter out ETFs without a 10-year ROI
filtered_roi_df = roi_df.dropna(subset=['10Y_ROI'])

# Sort by YTD ROI and then by 5-year ROI
sorted_roi_df = filtered_roi_df.sort_values(by=['YTD_ROI', '5Y_ROI'], ascending=[False, False])

print("Top ETFs based on YTD ROI and 5-year ROI:")
print(sorted_roi_df)


Top ETFs based on YTD ROI and 5-year ROI:
  Ticker   YTD_ROI    1Y_ROI    5Y_ROI   10Y_ROI  P/E Ratio
4   SPYG  3.335296  0.349041  1.229322  3.010477  31.576296
2    VOO  2.659634  0.290143  1.113357  2.402007  28.019978
3   SPLG  2.657562  0.289239  1.113521  2.427879  28.179306
1    IVV  2.655962  0.289812  1.112757  2.399403  28.224672
0    SPY  2.653823  0.288279  1.106906  2.398928  28.190062
7    RSP  1.887496  0.180806  0.821957  1.671751  21.894728
6   SPYV  1.865078  0.223104  0.902948  1.686024  24.673477
8    VXF  1.446263  0.213649  0.647414  1.351589  16.096668
9    IJR  1.430361  0.166861  0.645495  1.414782  14.087557
